In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

llm = ChatOpenAI()

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder( 
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "map_rerank",
    retriever = vectorstore.as_retriever()
)

chain.run("Describe Victory Mansions")

/Users/junghaein/Documents/FULLSTACK_GPT/env/lib/python3.11/site-packages/langchain/chains/llm.py:366: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
Error in LangChainTracer.on_chain_start callback: ValidationError(model='Run', errors=[{'loc': ('__root__',), 'msg': "argument of type 'NoneType' is not iterable", 'type': 'type_error'}])
Parent run cedf2d39-b6d5-4903-8517-580144dfcac0 not found for run aba984a3-2cd9-41f7-b82a-eddca6119ca9. Treating as a root run.
Parent run cedf2d39-b6d5-4903-8517-580144dfcac0 not found for run f131c3df-5445-483b-8271-90edd38affc1. Treating as a root run.
Parent run cedf2d39-b6d5-4903-8517-580144dfcac0 not found for run 9a45dbf2-f5e8-4ac5-b837-9c6232cddeff. Treating as a root run.
Parent run cedf2d39-b6d5-4903-8517-580144dfcac0 not found for run 93569058-b8f8-40ac-8070-7e95f0b7328e. Treating as a root run.
Error in LangChainTracer.on_chain_end callback: TracerException('No index

"Victory Mansions is a building with a smelly hallway, featuring boiled cabbage and old rag mats. The building is run-down, with a rarely working lift and posters of Big Brother watching you on every landing. The telescreen constantly blares out figures related to pig-iron production and cannot be fully turned off. Winston's flat is located on the seventh floor of the building."